# 1. Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np
from torchvision import datasets, models, transforms

from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim


import random
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split


from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification
from codes.utils import accuracy_classification
from codes.utils import cnn_class_cross_val_final_test

from codes.classification_codes import cnn_class_gridsearch


torch.backends.cudnn.deterministic = True
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Processed_Data2", "rb") as fp:   # Unpickling
  MoS2_data = pickle.load(fp)

df = pd.DataFrame(MoS2_data)
T = df['T']

T_classes = {900.0: 0, 950.0: 1, 1000.0: 2}

T_target = [T_classes[T[index]] for index, item in enumerate(T)]
sampleId = df['sampleId']
data_image = np.array([np.array(item) for item in df['image']])

print(len(T_target))

Data_CNN = data_image
print(Data_CNN.shape)
Data_CNN_rgb = np.repeat(Data_CNN[..., np.newaxis], 3, -1)
print(Data_CNN_rgb.shape)
Data_CNN_rgb = Data_CNN_rgb.transpose(0, 3, 1, 2)/255
print(Data_CNN_rgb.shape)


X = Data_CNN_rgb
Y = np.array(T_target)
#Y = Y.flatten()
groups = np.array(sampleId)

train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)


#Y = Y.flatten()



print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


def pretrained_model():
    
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

    model.fc = nn.Sequential(nn.ReLU(),
                          
                                    nn.Linear(512, 3) #150, 50176
                                   
                                     )
    model.to(device)
    return model

262
(262, 224, 224)
(262, 224, 224, 3)
(262, 3, 224, 224)
<class 'generator'>
(235, 3, 224, 224)
(235,)
(27, 3, 224, 224)
(27,)


# 3. Runing Cross-Validation

In [3]:
# the paramters below were optimized
# only the optimum parameters for the production model are used here

Learning_rate = [1e-4] #[1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 1e-5, 5e-6] 
#Drop_out = [0.2, 0.35, 0.5, 0.65, 0.75]
Batch_size = [32] #[24, 32, 100]

def cross_10_folds_cnn(train_val_X, train_val_Y):

    best_train = []
    best_val = []
    best_trainf1 = []
    best_valf1 =  []
    best_variables = []
    best_performance_record = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug3/CNN/{fold}_model.pth'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        #train_X, val_X, train_Y, val_Y = train_test_split(train_val_X, train_val_Y, test_size=0.1,stratify=train_val_Y,random_state=fold+30)#42
        best_train_acc, best_val_acc, trainf1, valf1, best_record, hyper=cnn_class_gridsearch(train_X, train_Y, val_X, val_Y, Learning_rate, Batch_size, fold, model_path)
        #best_train_acc, best_val_acc, trainf1, valf1, best_perf_record, hyper=cnn_class_train_fn(train_X, train_Y, val_X, val_Y, Learning_rate, Batch_size, fold)

        best_train.append(best_train_acc)
        best_val.append(best_val_acc)
        best_trainf1.append(trainf1)
        best_valf1.append(valf1)
        best_variables.append(hyper)
        best_performance_record.append(best_record)

        print(f'fold: {fold} done!')
    return best_train, best_val, best_trainf1, best_valf1,best_variables, best_performance_record


best_train, best_val, best_trainf1, best_valf1, best_variables, best_performance_record = cross_10_folds_cnn(train_val_X, train_val_Y)



<class 'generator'>
Epoch1: loss: 0.8901 val_loss: 1.1540
Epoch2: loss: 0.7929 val_loss: 0.9407
Epoch3: loss: 0.6567 val_loss: 0.8955
Epoch4: loss: 0.5437 val_loss: 0.9946
Epoch5: loss: 0.4724 val_loss: 1.0465
Epoch6: loss: 0.4935 val_loss: 1.0627
Epoch7: loss: 0.4376 val_loss: 0.8607
Epoch8: loss: 0.4018 val_loss: 1.1587
Epoch9: loss: 0.2632 val_loss: 1.2487
Epoch10: loss: 0.3014 val_loss: 0.9861
Epoch11: loss: 0.2754 val_loss: 1.2895
Epoch12: loss: 0.2450 val_loss: 1.2130
Epoch13: loss: 0.2990 val_loss: 1.3157
Epoch14: loss: 0.2988 val_loss: 1.1556
Epoch15: loss: 0.2757 val_loss: 1.2990
Epoch16: loss: 0.2728 val_loss: 0.8784
Epoch17: loss: 0.2427 val_loss: 1.6676
Epoch18: loss: 0.1958 val_loss: 1.4570
Epoch19: loss: 0.2202 val_loss: 1.6524
Epoch20: loss: 0.1583 val_loss: 1.1217
Epoch21: loss: 0.2076 val_loss: 1.3620
Epoch22: loss: 0.1744 val_loss: 1.3910
Epoch23: loss: 0.2433 val_loss: 1.0945
Early stopped training at epoch 23
Accuracy of the network on the 211 train images: 94.3 %
A

# 4. Test Models

In [4]:

trained_model = pretrained_model()
root_path = f'classification/ImageNet/aug3/CNN'

best_test, confusion_matrix_test = cnn_class_cross_val_final_test(trained_model, test_X, test_Y, 'test', root_path)


print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')
print("......")

def cm_to_plot(best_test):

    max_test = np.max(best_test)
    idx_cm = []
    for index, item in enumerate(best_test):
        if item == max_test:
            idx_cm.append(index)

    print('idx_cm: ', idx_cm)
    print('max_test_acc: ', max_test)

    print(confusion_matrix_test[idx_cm[0]])
cm_to_plot(best_test)
print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])


cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


Accuracy of the network on the 27 test images: 74.1 %
Accuracy of the network on the 27 test images: 70.4 %
Accuracy of the network on the 27 test images: 66.7 %
Accuracy of the network on the 27 test images: 59.3 %
Accuracy of the network on the 27 test images: 59.3 %
Accuracy of the network on the 27 test images: 63.0 %
Accuracy of the network on the 27 test images: 74.1 %
Accuracy of the network on the 27 test images: 70.4 %
Accuracy of the network on the 27 test images: 51.9 %
Accuracy of the network on the 27 test images: 70.4 %
[74.07407407407408, 70.37037037037037, 66.66666666666667, 59.25925925925926, 59.25925925925926, 62.96296296296296, 74.07407407407408, 70.37037037037037, 51.851851851851855, 70.37037037037037]
test_acc_mean: 65.926, std: 6.988
......
idx_cm:  [0, 6]
max_test_acc:  74.07407407407408
[[ 0  3  0]
 [ 1 11  1]
 [ 1  1  9]]
5th confusion matrix:  [[1, 2, 0], [2, 6, 5], [0, 2, 9]]
10
avg cm:  [[0.9, 1.9, 0.2], [1.8, 8.3, 2.9], [0.9, 1.5, 8.6]]
